# Práctica 3 de Recuperación de información: Analizadores y consultas con Whoosh 

<h7>(Milagros Fernández Gavilanes)</h7>

---

<div style="background-color:#d9f2d9; color:black; padding:15px; border-radius:8px; border:1px solid #5cb85c;">

En esta práctica vamos a seguir explorando la recuperación de información utilizando la librería <b>Whoosh</b>. <br>

Aprenderemos a:

<ul>
<li>1. Utilizar diferentes analizadores de texto.</li>
<li>2. Realizar consultas simples y con plugins.</li>
</ul>
Whoosh es una biblioteca de búsqueda en Python que permite construir índices y ejecutar consultas de manera rápida y flexible.

</div>

---
<div>
Primero, necesitamos tener instalado Whoosh en el entorno de Google Colab o de Jupyter Notebooks:
</div>

In [ ]:
%pip install Whoosh

<div style="background-color:#d9f2d9; color:black; padding:15px; border-radius:8px; border:1px solid #5cb85c;">
<h2 style="text-align:center;">Volviendo a la arquitectura de Whoosh</h2>

<h3>Habíamos dicho que los componentes básicos son:</h3>

<ul>
<li><b>Esquema (Schema)</b>: Define la <b>estructura de los documentos</b> que se van a indexar.   
  <ul>
    <li>Cada campo (por ejemplo <code>title</code> o <code>content</code>) tiene un tipo: <code>TEXT</code>, <code>ID</code>, <code>KEYWORD</code>, etc.</li>
    <li>Es el equivalente a un “modelo” de documento en Whoosh.</li>
  </ul>
</li>

<li><b>Fields</b>: son los <b>atributos</b> de cada campo del esquema del documento.  
Whoosh incluye tipos como:
  <ul>
    <li><code>ID</code>: identificador único, no analizado.</li>
    <li><code>TEXT</code>: texto analizado (tokenizado), usado para búsquedas.</li>
    <li><code>KEYWORD</code>: lista de términos separados por comas, útil para etiquetas.</li>
    <li><code>NUMERIC</code>: enteros o decimales indexables.</li>
    <li><code>DATETIME</code>: fecha y hora.</li>
  </ul>
  Permite especificar cómo se procesa el contenido, por ejemplo tokenización, normalización o análisis de texto.
</li><br>

<li><b>Análisis de texto (Analyzer)</b>: define cómo se transforma el texto. Los analizadores procesan el texto de los documentos y las consultas: tokenización, minúsculas, eliminación de stopwords, stemming, etc. Es plugable, lo que significa que puedes personalizar cómo se interpreta el texto. Whoosh trae analizadores como:
  <ul>
  <li><code>StandardAnalyzer</code> (estándar): El más común. Divide el texto en tokens (palabras), pasa a minúsculas y elimina stopwords (palabras vacías como “el”, “la”, “y”).</li>
  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import StandardAnalyzer
analyzer = StandardAnalyzer()</pre>
  </div>
  <li><code>SimpleAnalyzer</code>: Divide el texto en tokens por caracteres alfanuméricos y pasa todo a minúsculas. No elimina stopwords.</li>

  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import SimpleAnalyzer
analyzer = SimpleAnalyzer()</pre>
</div>
  <li><code>KeywordAnalyzer</code>: Trata todo el texto como un solo token. Útil para campos que no deben dividirse (ejemplo: códigos, tags).</li>
  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import KeywordAnalyzer
analyzer = KeywordAnalyzer()</pre>
  </div>
  <li><code>RegexAnalyzer</code>: Permite definir un patrón de expresión regular para tokenizar.</li>
  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import RegexAnalyzer
analyzer = RegexAnalyzer(r"\w+")</pre>
  </div>
  <li>Analizadores avanzados:
  <ul>
  <li><code>StemmingAnalyzer</code>: Aplica un algoritmo de stemming (reduce palabras a su raíz, ej. “corriendo” → “correr”). Mejora la recuperación al agrupar variaciones de palabras.</li>
  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import StemmingAnalyzer
analyzer = StemmingAnalyzer()</pre>
  </div>
  <li>Analizadores para lenguajes específicos (con <code>SnowballStemmer</code>): Para diferentes idiomas (español, inglés, francés, etc.).</li>
  <div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">from whoosh.analysis import StemmingAnalyzer
from whoosh.lang.snowball import SpanishStemmer
analyzer = StemmingAnalyzer(stemfn=SpanishStemmer())</pre>
  </div>
  </ul>
</ul>
</li>
<li><b>Consulta (Query)</b>: El parser de consultas convierte las consultas de texto en objetos <code>Query</code>.
<ul>
  <li>Soporta operadores booleanos, búsqueda por proximidad, coincidencia aproximada y combinaciones complejas.</li>
  <li>El motor de búsqueda ejecuta la consulta contra el índice invertido para obtener los documentos relevantes.</li>
</ul>
</li>
</ul>

---
</div>


<div style="background-color:#333333; color:white; padding:15px; border-radius:8px; border:1px solid #000;">

<h2 style="margin-top:0; color:white;">Guión</h2>

<b>Una vez importadas las librerías, nuestro punto de partida debería ser el esquema proporcionado la semana pasada, con el fin de especificar estos campos de los documentos en un índice. Sin embargo, comenzaremos definiendo el conjunto de documentos, junto con su estructura (que hará uso del esquema).</b>
<br>

<b>Nota:</b> El esquema es el conjunto de todos los campos posibles en un documento.

Cada documento individual puede usar solo un subconjunto de los campos disponibles en el esquema.

</div>


<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">En primer lugar importamos las librerías...</span>
</div>


In [ ]:
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, NUMERIC
from whoosh.index import create_in
from whoosh import analysis
import os

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">En segundo lugar definimos los documentos que usaremos más adelante ... Se trata de tres libros ...</span>
</div>


In [ ]:
abstract1 = """Captain Alatriste, by Arturo Pérez-Reverte, narrates the adventures of a veteran soldier in 17th-century Spain during the Habsburg era. Alatriste is a swordsman and mercenary who faces conspiracies, duels, and dangers while adhering to a strict code of honor. The series combines action, historical intrigue, and daily life in Spain’s Golden Age, offering an intense and realistic portrayal of the period. Magic and loyalty play a subtle but important role in Alatriste's tale."""

abstract2 = """Children's and Household Tales (German: Kinder- und Hausmärchen) is a collection of fairy tales first published on 20 December 1812 by the Grimm brothers, Jacob and Wilhelm. The series is commonly known in English as Grimms' Fairy Tales. Many stories involve children, magic, and lessons about loyalty and courage."""

abstract3 = """Harry Potter and the Half-Blood Prince, by J.K. Rowling, follows Harry’s sixth year at Hogwarts School of Witchcraft and Wizardry. As Voldemort’s power grows stronger, Harry discovers a mysterious potions textbook once owned by the “Half-Blood Prince,” which helps him excel in class while revealing secrets about Voldemort’s past. Together with Dumbledore, Harry learns about the dark lord’s Horcruxes, setting the stage for the final confrontation. Themes of magic, loyalty, and courage are central throughout the story."""

# Lista de documentos de ejemplo

docs = [
    {
        "year": "1996",
        "author": "Arturo Pérez-Reverte",
        "title": "Captain Alatriste",
        "abstract": abstract1,
        "subject": "novel, historic",
        "keywords": "Captain Alatriste, Habsburg era, Mercenary",
    },
    {
        "year": "1812",
        "author": " Jacob and Wilhelm",
        "title": "Grimms' Fairy Tales",
        "abstract": abstract2,
        "subject": "story, children",
        "keywords": "The Frog King,  Rapunzel",
    },
    {
        "year": "2006",
        "author": "Joanne Rowling",
        "title": "Harry Potter",
        "abstract": abstract3,
        "subject": "novel, fantasy",
        "keywords": "Harry Potter, Half-Blood Prince, Magic",
    },
]

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Lo que vamos a hacer a continuación es crear un índice utilizando diferentes analizadores y luego veremos cómo afectan los resultados de búsqueda.</span>

<span style="color:black">El punto de partida será la creación de la carpeta que contendrá los índices invertidos y a continuación crearemos el índice pero con una particularidad: este <b>índice con diferentes analizadores</b>.</span>
</div>


In [ ]:
def create_index(name, analizador):
    schema = Schema(
        year=NUMERIC(stored=True),
        author=TEXT(analyzer=analizador, stored=True),
        title=TEXT(analyzer=analizador, stored=True),
        abstract=TEXT(analyzer=analizador, stored=True),
        subject=KEYWORD(commas=True, scorable=True),
        keywords=KEYWORD(commas=True, scorable=True),
    )

    # Crear carpeta para el índice
    dir_name = "indexdir_" + name
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

    ix = create_in(dir_name, schema)

    writer = ix.writer()
    for doc in docs:
        writer.add_document(
            year=doc["year"],
            author=doc["author"],
            title=doc["title"],
            abstract=doc["abstract"],
            subject=doc["subject"],
            keywords=doc["keywords"],
        )
    writer.commit()
    return ix


<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Estos son los tipos de campos predefinidos utilizados:<br>

<ul>
    <li>1. <code>whoosh.fields.NUMERIC</code>:<br>
            Este campo almacena números enteros (<code>int</code>), largos (<code>long</code>) o de coma flotante (<code>float</code>) en un formato compacto y ordenable.</li><br>
    <li>2.<code>whoosh.fields.TEXT</code>:<br>
            Los campos <code>TEXT</code> pueden indexar el texto y almacenar posiciones de términos (por defecto, <code>TEXT(phrase=True)</code>) para permitir la <b>búsqueda por frases</b>.</li><br>
    <li>3.<code>whoosh.fields.KEYWORD</code>:<br>
        Este tipo de campo está diseñado para palabras clave separadas por espacios o comas. Este tipo se indexa y es buscable (y opcionalmente almacenable).
    Sin embargo, no admite búsquedas de frases.<br>
    Además convierte automáticamente a minúsculas las palabras clave antes de indexarlas, usando <code>lowercase=True</code>.<br>
    Para separar las palabras clave con comas (lo que permite palabras clave que contengan espacios), usa <code>commas=True</code>. De lo contrario, las palabras clave se separan por espacios.<br><br>
    Ahora bien, ¿qué significa que esté activo la opción <code>scorable=True</code>?<br><br>
    <ul>
        <li>Si <code>scorable=False</code> (por defecto), las búsquedas en ese campo solo devuelven coincidencias exactas, pero no asignan un score (puntuación de relevancia). Todos los documentos coincidentes se consideran “iguales”.</li>
        <li>Si <code>scorable=True</code>, las búsquedas en ese campo sí generan puntuaciones de relevancia, de manera que los documentos que contengan más coincidencias (o más fuertes) aparecen más arriba en el ranking de resultados. </li>
    </ul></li>
</ul>

Nota: existen muchos otros campos predefinidos que los usuarios pueden elegir.
Consulta la documentación en: http://whoosh.readthedocs.io/en/latest/api/fields.html#pre-made-field-types.</span>
</div>



<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Procedemos definir los analizadores y a crear cada uno de estos índices...</span>
</div>


In [ ]:
analyzers = {
    "StandardAnalyzer": analysis.StandardAnalyzer(),
    "RegexAnalyzer": analysis.RegexAnalyzer(),
    "StemmingAnalyzer": analysis.StemmingAnalyzer(),
    "SimpleAnalyzer": analysis.SimpleAnalyzer(),
    "FancyAnalyzer": analysis.FancyAnalyzer(),
    "LanguageAnalyzer": analysis.LanguageAnalyzer("en"),
}

indices = {}
for name, analyzer in analyzers.items():
    ix = create_index(name, analyzer)
    print(f"Indice vacío para {name} creado")
    indices[name] = ix

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Definimos una función interactiva para probar consultas ...<br>

Después de indexar los documentos, podemos escribir la consulta y convertir la cadena de consulta en un objeto de consulta usando el query parser.<br>

Para ello, creamos un objeto <code>whoosh.qparser.QueryParser</code>, al cual le pasamos el nombre del campo por defecto donde se realizará la búsqueda y el esquema del índice en el que se consultará.<br>

<b>Nota:</b> ¡Muy importante! La cadena de consulta debe ser un valor Unicode.
</span>
</div>


In [ ]:
def search_query(analyzer_name, field, plugin, query_text):
    ix = indices[analyzer_name]
    with ix.searcher() as searcher:
        qp = QueryParser(field, ix.schema)
        if plugin is not None:
            qp.add_plugin(plugin)
        query = qp.parse(query_text)

        results = searcher.search(query)
        print(
            f"--- Analizador: {analyzer_name} | Campo: {field} | Consulta: '{query_text}' y sus \"tokens\" '{query}'---"
        )
        if results:
            for r in results:
                print(f"{r['title']} ({r['year']})")
        else:
            print("No results")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Definimos una función interactiva para ver tokens generados por cada analizador ...</span>
</div>

In [ ]:
def show_tokens(analyzer_name, text):
    analyzer = analyzers[analyzer_name]
    print(f"--- Tokens with {analyzer_name} ---")
    for token in analyzer(text):
        print(token.text)

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Si queremos que los términos de la búsqueda se busquen tanto en el título como en el abstract, se utilizaría utiliza un <code>whoosh.qparser.MultifieldParser</code>
<div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">qp = MultifieldParser(["title", "abstract"], schema=schema)
query = qp.parse(query_text)</pre>
</div>
</span>
</div>


<div style="background-color:#fff3cd; color:#665200; padding:15px; border-radius:8px; border:1px solid #ffecb5;">

<h3 style="margin-top:0; color:#665200;">Trabajo a realizar:</h3>

1. Crear otra función search_query2 que haga uso de <code>whoosh.qparser.MultifieldParser</code>.

2. Añadir líneas de código realizando las mismas consultas que lo que viene a continuación.


</div>

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Ejemplo que funciona con todos los analizadores:</span>
</div>

In [ ]:
# Buscamos todos los que tengan "Joanne Rowling" en author
search_query("StandardAnalyzer", "author", None, "Joanne Rowling")
search_query("RegexAnalyzer", "author", None, "Joanne Rowling")
search_query("StemmingAnalyzer", "author", None, "Joanne Rowling")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Ejemplo que funciona solo con algunos analizadores:</span>
</div>

In [ ]:
# Buscamos todos los que tengan "Rowl" en author
search_query("StandardAnalyzer", "author", None, "Rowl")
search_query("RegexAnalyzer", "author", None, "Rowl")
search_query("StemmingAnalyzer", "author", None, "Rowl")

In [ ]:
# Buscamos todos los que tengan "collection AND tales AND story" en abstract
search_query("StandardAnalyzer", "abstract", None, "collection tales story")
search_query("RegexAnalyzer", "abstract", None, "collection tales story")
search_query("StemmingAnalyzer", "abstract", None, "collection tales story")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Vamos a ver los tokens generados por cada analizador según los ejemplos:</span>
</div>

In [ ]:
# Ejemplo: tokenización de "Joanne Rowling"
show_tokens("StandardAnalyzer", "Joanne Rowling")
show_tokens("RegexAnalyzer", "Joanne Rowling")
show_tokens("StemmingAnalyzer", "Joanne Rowling")

In [ ]:
# Ejemplo: tokenización de "Rowl"
show_tokens("StandardAnalyzer", "Rowl")
show_tokens("RegexAnalyzer", "Rowl")
show_tokens("StemmingAnalyzer", "Rowl")

In [ ]:
# Ejemplo: tokenización de "collection tales story"
show_tokens("StandardAnalyzer", "collection tales story")
show_tokens("RegexAnalyzer", "collection tales story")
show_tokens("StemmingAnalyzer", "collection tales story")

In [ ]:
# Buscamos todos los que tengan "collection OR (own confrontation)" en abstract
search_query("StandardAnalyzer", "abstract", None, "collection OR (own confrontation)")
search_query("RegexAnalyzer", "abstract", None, "collection OR (own confrontation)")
search_query("StemmingAnalyzer", "abstract", None, "collection OR (own confrontation)")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Por defecto, el parser trata las palabras como si estuvieran conectadas por AND. Si no queremos tener que indicarlo podemos cambiar el argumento "group" si queremos que estén conectadas por OR.
</span>
</div>

<div style="background-color:#fff3cd; color:#665200; padding:15px; border-radius:8px; border:1px solid #ffecb5;">

<h3 style="margin-top:0; color:#665200;">Trabajo a realizar:</h3>

1. Crear otra función search_query3 que utilice:
<div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">qp = MultifieldParser(["title", "abstract"], schema=schema, group=OrGroup)
query = qp.parse(query_text)</pre>
</div>

2. Añadir líneas de código realizando las mismas consultas que las anteriores.


</div>



<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">El query parser está construido sobre complementos modulares (plug-ins).<br>

Por ejemplo, <code>GtLtPlugin()</code> te permite usar <code>>, <, >=, <=, => o =<</code> después de especificar el campo, y traduce la expresión al rango equivalente:
<div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">qp.add_plugin(GtLtPlugin()) 
query=qp.parse(u"year:<2000") </pre>
</div>

A continuación, vemos un ejemplo con GtLtPlugin:</span>
</div>

In [ ]:
# Buscamos todos los que se hayan publicado antes del 2000
search_query("StandardAnalyzer", "year", GtLtPlugin(), "year:<2000")
search_query("RegexAnalyzer", "year", GtLtPlugin(), "year:<2000")
search_query("StemmingAnalyzer", "year", GtLtPlugin(), "year:<2000")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">En este caso no importa qué analizador uses (StandardAnalyzer, RegexAnalyzer, StemmingAnalyzer) porque el campo year es numérico (NUMERIC) y los analizadores solo afectan campos de texto (TEXT).

<ul>
    <li>Los analizadores solo tokenizan y transforman texto; no tienen efecto sobre búsquedas numéricas.</li>
    <li>Lo importante para búsquedas con GtLtPlugin es que el plugin esté agregado antes de parsear la consulta y que la sintaxis sea correcta <code>(<2000, >1990, etc.)</code>.</li>
</ul></span>
</div>



<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Otro ejemplo, <code>FuzzyTermPlugin()</code> te permite buscar términos “difusos”, es decir, términos que no tienen que coincidir exactamente. El término difuso coincidirá con cualquier término similar dentro de un cierto número de “ediciones” (cambios):
<div style="text-align:center; margin:10px;">
    <pre style="display:inline-block; text-align:left; border:1px solid #990000; border-radius:6px; padding:10px; background-color:#fff;">qp.add_plugin(FuzzyTermPlugin()) 
query=qp.parse(u"author:revert~") </pre>
</div>

A continuación, vemos un ejemplo con FuzzyTermPlugin:</span>
</div>

In [ ]:
# Buscamos todos los autores que tenga "revert" y todos los términos en el índice que estén a una “edición”, por ejemplo "reverte" insertando una "e".
search_query("StandardAnalyzer", "author", FuzzyTermPlugin(), "revert~")
search_query("RegexAnalyzer", "author", FuzzyTermPlugin(), "revert~")
search_query("StemmingAnalyzer", "author", FuzzyTermPlugin(), "revert~")

<div style="background-color: lightblue; padding: 10px; border-radius: 5px;">
<span style="color:black">Otro ejemplo, <code>qparser.WildcardPlugin</code>, que ya se incluye en la lista de complementos por defecto del parser, le da la capacidad de realizar búsquedas con comodines (wildcards). Algunos complementos de uso frecuente se muestran en el siguiente código.<br>

Puedes usar el argumento plugins al crear el objeto para sobrescribir la lista de complementos por defecto, o bien usar los métodos add_plugin() y/o remove_plugin_class() para modificar los complementos incluidos en el parser.<br>

Aquí está la lista de complementos disponibles: http://whoosh.readthedocs.io/en/latest/api/qparser.html#plug-ins<br>

A continuación, vemos un ejemplo con wildcard:</span>
</div>

In [ ]:
# Buscamos todos los que tengan "Harry*" en keywords
search_query("StandardAnalyzer", "keywords", None, "Harry*")
search_query("RegexAnalyzer", "keywords", None, "Harry*")
search_query("StemmingAnalyzer", "keywords", None, "Harry*")

In [ ]:
# Buscamos todos los que tengan "Harry*" en keywords
search_query("StandardAnalyzer", "abstract", None, '"magic loyalty"~2')
search_query("RegexAnalyzer", "abstract", None, '"magic loyalty"~2')
search_query("StemmingAnalyzer", "abstract", None, '"magic loyalty"~2')

<div style="background-color:#fff3cd; color:#665200; padding:15px; border-radius:8px; border:1px solid #ffecb5;">

<h3 style="margin-top:0; color:#665200;">Trabajo a realizar:</h3>

1. ¿Para qué sirve RegexAnalyzer?

2. Prueba a poner ejemplos cambiando la configuración de RegexAnalyzer.


</div>
